In [1]:
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex
import os

from dotenv import load_dotenv

load_dotenv()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.9.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


True

# helper functions

In [2]:
## embedding model
def get_embedding_model(model_name="/teamspace/studios/this_studio/bge-small-en-v1.5"):
    embed_model = HuggingFaceEmbedding(model_name=model_name)
    return embed_model

# generator model
def get_llm(model_name="llama3-8b-8192"):
    llm = Groq(model=model_name, api_key=os.getenv("GROQ_API"), temperature=0.8)
    return llm

## get deeplake vector database
def get_vector_database(id, dataset_name):
    my_activeloop_org_id = id # "hunter"
    my_activeloop_dataset_name = dataset_name # "Vietnamese-law-RAG"
    dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
    vector_store = DeepLakeVectorStore(
        dataset_path=dataset_path,
        overwrite=False,
    )
    return vector_store

def get_index(vector_store):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    return index

In [3]:
## query generation / rewriting
from llama_index.core import PromptTemplate

query_str = "Vượt đèn đỏ sẽ bị gì?"

prompt_for_generating_query = PromptTemplate(
"""Bạn là một trợ lý xuất sắc trong việc tạo ra các câu truy vấn tìm kiếm liên quan. Dựa trên câu truy vấn đầu vào dưới đây, hãy tạo ra {num_queries} truy vấn tìm kiếm liên quan, mỗi câu trên một dòng. Lưu ý, trả lời bằng tiếng Việt và chỉ trả về các truy vấn đã tạo ra.

### Câu truy vấn đầu vào: {query}

### Các câu truy vấn:"""
)

def generate_queries(llm, query_str, num_queries=4):
    fmt_prompt = prompt_for_generating_query.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries =  response.text.split("\n")
    return queries

def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.retrieve(query))
    
    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, tasks)):
        results_dict[(query, i)] = query_result
    
    return results_dict

In [4]:
from llama_index.retrievers.bm25 import BM25Retriever

def get_bm25_retriever(index, similarity_top_k=13625):
    source_nodes = index.as_retriever(similarity_top_k=similarity_top_k).retrieve("test")
    nodes = [x.node for x in source_nodes]
    bm25 = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=3)
    return bm25

# advanced RAG

- generate 5 queries related to user queries
- retrieved nodes with two search methods: vector search and bm25 -> only retain nodes with score > 0.75
- metadata replacement + rerank
- queries

In [5]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama

## set embedding model
embed_model = get_embedding_model()
Settings.embed_model = embed_model

## set llm 
llm = Ollama(model="vistral", request_timeout=120.0, max_new_tokens=2000) # get_llm()
Settings.llm = llm

## get vector store, index, and two retrievers
vector_store = get_vector_database("hunter", "Vietnamese-legal-data")
index = get_index(vector_store=vector_store)
vector_retriever = index.as_retriever(similarity_top_k=3)
bm25_retriever = get_bm25_retriever(index)

Deep Lake Dataset in hub://hunter/Vietnamese-legal-data already exists, loading from the storage


**Gen user queries**

In [6]:
user_query = "Tôi muốn mở nhà hàng thì cần thực hiện những bước nào?"

In [7]:
from llama_index.llms.gemini import Gemini

queries = generate_queries(llm=Gemini(), query_str=user_query, num_queries=5)

NotFound: 404 Gemini 1.0 Pro Vision has been deprecated on July 12, 2024. Consider switching to different model, for example gemini-1.5-flash.

In [8]:
print(queries)

['- Các bước mở nhà hàng cần thực hiện', '- Quy trình mở nhà hàng chi tiết', '- Hướng dẫn mở nhà hàng từ A đến Z', '- Thủ tục mở nhà hàng mới nhất']


**retrieve all nodes**

In [8]:
retrievers = [vector_retriever, bm25_retriever]

In [9]:
nodes = []
for retriever in retrievers:
    for query in queries:
        retrieved_nodes = retriever.retrieve(query)
        for node in retrieved_nodes: 
            if node.score >= 0.7:
                nodes.append(node)
    
    # retrieve nodes for original query
    retrieved_nodes = retriever.retrieve(user_query)
    for node in retrieved_nodes: 
        if node.score >= 0.7:
            nodes.append(node)

print(len(nodes))

NameError: name 'queries' is not defined

**Metadata replacement**

In [10]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

replacement = MetadataReplacementPostProcessor(target_metadata_key="window")
nodes = replacement.postprocess_nodes(nodes)

**rerank using Cohere**

In [11]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(model="rerank-multilingual-v2.0", api_key=os.getenv('COHERE_API_KEY'), top_n=3)  # remain top 3 relevant

from llama_index.core.schema import QueryBundle
final_nodes = cohere_rerank.postprocess_nodes(
    nodes, QueryBundle(user_query)
)

**Query**

In [12]:
prompt_ = """
Bạn là một trợ lý ảo về tư vấn pháp luật. Nhiệm vụ của bạn là sinh ra câu trả lời dựa vào hướng dẫn được cung cấp, kết hợp thông tin từ tài liệu tham khảo với khả năng suy luận và kiến thức chuyên môn của bạn để đưa ra câu trả lời sâu sắc và chi tiết.
Ví dụ: Nếu văn bản được truy xuất nói về một điểm pháp luật, nhưng câu hỏi liên quan đến một tình huống thực tế, bạn cần dựa vào thông tin đó để giải quyết hoặc trả lời thấu đáo câu hỏi.
# Quy tắc trả lời:
1. Kết hợp thông tin từ phần tài liệu tham khảo ## context với khả năng suy luận và kiến thức chuyên môn của bạn để đưa ra câu trả lời chi tiết và sâu sắc.
2. Trả lời như thể đây là kiến thức của bạn, không dùng các cụm từ như: "dựa vào thông tin bạn cung cấp", "dựa vào thông tin dưới đây", "dựa vào tài liệu tham khảo",...
3. Từ chối trả lời nếu câu hỏi chứa nội dung tiêu cực hoặc không lành mạnh.
4. Trả lời với giọng điệu tự nhiên và thoải mái như một chuyên gia thực sự.
# Định dạng câu trả lời:
1. Câu trả lời phải tự nhiên và không chứa các từ như: prompt templates, ## context...
2. Không cần lặp lại câu hỏi trong câu trả lời.
3. Hãy trả lời theo format dễ nhìn, xuống dòng khi cần thiết

----------------------
## content: 
{context_str}

## user query:
{query_str}

## Trả lời:
"""

prompt_ = PromptTemplate(prompt_)

In [13]:
llm = Ollama(model="vistral", request_timeout=120.0, max_new_tokens=4000) # get_llm()
Settings.llm = llm

In [14]:
context = "\n\n".join([node.get_content() for node in final_nodes])
print(llm.complete(
    prompt_.format(
        query_str=user_query,
        context_str=context
    )
))


Trước hết, bạn phải làm thủ tục thành lập hộ kinh doanh hoặc công ty. Để được phép hoạt động trong lĩnh vực này, bạn phải xin cấp giấy chứng nhận đủ điều kiện an toàn vệ sinh thực phẩm từ cơ quan nhà nước có thẩm quyền theo quy định của pháp luật về vệ sinh an toàn thực phẩm. Ngoài ra, căn cứ Điều 37 Nghị định số 106/2015/NĐ-CP ngày 28 tháng 10 năm 2015 của Chính phủ hướng dẫn thi hành một số điều của Luật Bảo hiểm xã hội về bảo hiểm xã hội bắt buộc thì chủ doanh nghiệp phải có trách nhiệm đóng các khoản bảo hiểm sau đây cho người lao động: - Bảo hiểm xã hội. - Bảo hiểm y tế. - Bảo hiểm thất nghiệp. Đối với trường hợp hộ kinh doanh cá thể không sử dụng thường xuyên từ đủ 10 lao động trở lên thì bạn sẽ nộp hồ sơ tại cơ quan Bảo hiểm xã hội cấp huyện nơi đặt trụ sở của hộ kinh doanh; đối với các hộ kinh doanh cá thể sử dụng thường xuyên từ đủ 10 lao động trở lên thì nộp hồ sơ tại cơ quan bảo hiểm xã hội tỉnh. Về nội dung câu hỏi, bạn cần cung cấp thêm thông tin chi tiết để được trả lời 

In [15]:
q = index.as_query_engine()
q.get_prompts()["response_synthesizer:text_qa_template"]

SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x7f76dc8748b0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content="You are an exp

In [16]:
print(q.get_prompts()["response_synthesizer:text_qa_template"].get_template())

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


# Put everything together

In [17]:
def answer(user_query):
    # gen queries
    queries = generate_queries(llm=Gemini(), query_str=user_query, num_queries=5)

    ## get relevant nodes
    nodes = []
    for retriever in retrievers:
        for query in queries:
            retrieved_nodes = retriever.retrieve(query)
            for node in retrieved_nodes: 
                if node.score >= 0.7:
                    nodes.append(node)
        
        # retrieve nodes for original query
        retrieved_nodes = retriever.retrieve(user_query)
        for node in retrieved_nodes: 
            if node.score >= 0.7:
                nodes.append(node)
    
    ## metadata replacement
    nodes = replacement.postprocess_nodes(nodes)

    ## rerank 
    final_nodes = cohere_rerank.postprocess_nodes(
        nodes, QueryBundle(user_query)
    )

    ## query
    context = "\n".join([node.get_content() for node in final_nodes])
    response = llm.complete(
        prompt_.format(query_str=user_query, context_str=context)
    )

    return response

In [18]:
query1 = "Giấy xác nhận nhân thân là gì?"
response1 = answer(query1)

NotFound: 404 Gemini 1.0 Pro Vision has been deprecated on July 12, 2024. Consider switching to different model, for example gemini-1.5-flash.

In [20]:
print(response1)


Theo quy định tại điểm a khoản 2 Điều 10 Thông tư số 67/2015/TT-BCA, giấy xác nhận nhân thân được cấp cho cá nhân khi có yêu cầu để sử dụng vào mục đích nhập học hoặc các công việc cần thiết khác. Giấy này gồm một trang, do Trưởng Công an xã, phường, thị trấn nơi người đó cư trú thường xuyên cấp theo mẫu quy định tại Phụ lục 01 ban hành kèm theo Thông tư số 67/2015/TT-BCA. Khi cá nhân yêu cầu xác nhận về tình trạng hôn nhân thì cơ quan đăng ký hộ tịch có thẩm quyền giải quyết và ghi nội dung liên quan đến tình trạng hôn nhân của người đó vào Giấy xác nhận nhân thân (kể cả trường hợp trước đây đã từng ly hôn hoặc đang trong thời kỳ chờ giải quyết ly hôn). 


In [21]:
query2 = "Không được sự đồng ý của gia đình tôi, hàng xóm vẫn trổ cửa sổ. Nếu tôi muốn kiện ra tòa án thì thủ tục như thế nào? Án phí bao nhiêu? Thời gian tiến hành bao lâu?"
response2 = answer(query2)
print(response2)

 
Hàng xóm xây dựng công trình không đúng quy định pháp luật và gây ảnh hưởng đến quyền sử dụng đất của gia đình bạn. Theo đó, bạn có thể làm đơn gửi tới Ủy ban nhân dân cấp xã để giải quyết vì việc xây dựng thuộc thẩm quyền xử lý hành chính của UBND xã. Trường hợp UBND xã đã lập biên bản nhưng hàng xóm vẫn không khắc phục thì bạn có thể tiến hành khởi kiện tại Toà án theo quy định pháp luật tố tụng dân sự và Bộ luật Tố tụng Dân sự 2015. Chi phí giải quyết sẽ do người thua kiện chịu trách nhiệm chi trả, ngoài ra bạn còn phải nộp tiền tạm ứng án phí để được giải quyết vụ việc. Thời gian giải quyết có thể kéo dài từ vài tháng đến một năm tùy thuộc vào tính chất phức tạp của tranh chấp và lượng công việc xét xử tại Toà án cấp huyện. 


In [112]:
query3 = "Theo quy định của pháp luật hiện hành thì khi vợ chồng ly hôn, người vợ có được thay đổi họ của người con trên giấy khai sinh không?"
response3 = answer(query3)
print(response3)


Khi cha mẹ đã ly hôn mà muốn thay đổi thông tin trên Giấy khai sinh cho con (họ và tên) hoặc chuyển quyền nuôi con cần làm thủ tục như sau: 1. Lập hồ sơ đăng ký thay đổi, cải chính hộ tịch gồm những giấy tờ sau đây : a) Tờ khai theo mẫu quy định; b) Bản chính Giấy khai sinh của người có yêu cầu thay đổi, cải chính hộ tịch; c) Văn bản chứng minh việc thay đổi họ tên cho con (ví dụ Quyết định ly hôn); d) Chứng minh nhân dân hoặc thẻ căn cước công dân của cha mẹ. 2. Nộp hồ sơ tại cơ quan quản lý nhà nước về đăng ký hộ tịch, nơi đã cấp Giấy khai sinh cho người có yêu cầu thay đổi, cải chính hộ tịch. 3. Trong thời hạn 03 ngày làm việc kể từ ngày nhận đủ giấy tờ theo quy định, nếu thấy nội dung đề nghị phù hợp với quy định của pháp luật và thủ tục thì Sở Tư pháp hoặc Phòng tư pháp sẽ thực hiện việc thay đổi, cải chính hộ tịch cho công dân. Trường hợp cần phải xác minh thì thời hạn giải quyết không quá 05 ngày làm việc. 4. Khi đã nhận được kết quả, cha mẹ có thể đến UBND cấp xã để xin trích 

In [115]:
query4 = "Hợp đồng vay 10 triệu có buộc phải công chứng không?"
response4 = answer(query4)
print(response4)


Điều 38 Luật Công chứng năm 2014 quy định về việc ký, đóng dấu của tổ chức hành nghề công chứng như sau: "Người yêu cầu công chứng phải ký vào từng trang của hợp đồng, giao dịch. Tổ chức hành nghề công chứng phải ghi lời chứng, ngày tháng năm và đóng dấu vào văn bản." Điều này có nghĩa là chỉ cần người yêu cầu công chứng ký tên mình thì hợp đồng sẽ được công chứng mà không phân biệt giá trị tiền bạc trong hợp đồng đó. 

Theo quy định tại Nghị quyết số 50/2019 của Hội đồng Thẩm phán Tòa án nhân dân tối cao, các văn bản dưới đây là những trường hợp bắt buộc phải có chứng thực chữ ký: 
- Hợp đồng mua bán, tặng cho, đổi nhà ở; 
- Văn bản về thừa kế nhà ở và quyền sử dụng đất ở. 
Trường hợp của bạn không thuộc diện này nên việc công chứng hay không phụ thuộc vào nhu cầu của các bên tham gia giao dịch. Tuy nhiên nếu có tranh chấp phát sinh thì việc công chứng sẽ là một căn cứ quan trọng để Tòa án giải quyết theo quy định pháp luật. Ngoài ra, cũng cần lưu ý rằng người nhận tiền vay phải chịu

In [21]:
query5 = "Người làm nhục người khác bồi thường thiệt hại cho người bị xâm phạm như thế nào?"
response5 = answer(query5)
print(response5)


Để hạn chế, khắc phục thiệt hại xảy ra khi danh dự, nhân phẩm, uy tín của một người bị xâm phạm thì phải chi phí hợp lý để hạn chế, khắc phục thiệt hại đó. Ngoài ra, trong trường hợp này, người chịu trách nhiệm bồi thường còn phải bồi thường một khoản tiền khác để bù đắp tổn thất về tinh thần cho những người thân thích thuộc hàng thừa kế thứ nhất của người bị thiệt hại nếu không có những người này thì người mà người bị thiệt hại đã trực tiếp nuôi dưỡng hoặc đã trực tiếp nuôi dưỡng người bị thiệt hại được hưởng khoản tiền này. Mức bồi thường bù đắp tổn thất về tinh thần do các bên thỏa thuận; nếu không thỏa thuận được thì mức tối đa cho một người có danh dự, nhân phẩm, uy tín bị xâm phạm không quá mười lần mức lương cơ sở do Nhà nước quy định. 


In [22]:
query6 = "Dì em đứng tên chủ sở hữu 01 căn nhà (có nguồn gốc do bố mẹ chồng dì để lại). Nay dì em đã chết. Xin hỏi: căn nhà dì em đứng tên được xác định là tài sản chung hay riêng của vợ chồng? Dì em có 1 con gái duy nhất (10 tuổi). Chồng dì có vợ hai, có con riêng nhưng hiện không biết đứa trẻ đó đang ở đâu. Dì em mất đi không để lại di chúc thì tài sản là căn nhà trên sẽ được chia như thế nào? Chồng dì do uống rượu say nên ngã xe bị tai biến. Con dì có nguyện vọng ở với gia đình em thì thủ tục giám hộ như thế nào? Em chân thành cám ơn!"
response6 = answer(query6)
print(response6)

 
Căn cứ theo quy định tại Điều 274 Bộ luật Dân sự năm 2015 về nghĩa vụ, trong trường hợp dì của bạn đã chết không để lại di chúc thì căn nhà do dì đứng tên chủ sở hữu được xác định là tài sản riêng của người đó và sẽ được chia thừa kế cho hàng thừa kế theo quy định tại Điều 651 Bộ luật Dân sự năm 2015. Cụ thể, trong trường hợp này con gái (con đẻ) của dì bạn được hưởng phần di sản bằng hai phần ba một suất của người thừa kế theo pháp luật nếu dì bạn có cha mẹ ruột còn sống; hoặc bằng hai phần ba giá trị phần di sản mà dì bạn để lại nếu dì không còn cha mẹ ruột và dì có vợ, chồng thì tài sản chung giữa dì bạn với vợ/chồng sẽ được chia đều cho cả hai. Trường hợp con gái (con riêng) của chồng dì bạn cũng là hàng thừa kế theo pháp luật nhưng do đứa bé này hiện tại chưa đủ tuổi để hưởng di sản nên phần di sản mà đáng lẽ ra đứa trẻ đó phải được nhận sẽ được giao lại cho người quản lý tài sản của đứa trẻ để chăm sóc, nuôi dưỡng. 


In [23]:
query6 = "Vợ chồng tôi cưới nhau đã 7 năm nay nhưng vẫn chưa có con. Vừa qua có người giới thiệu cho chúng tôi xin nhận một cháu làm con nuôi. Xin luật gia hướng dẫn các thủ tục?"
response6 = answer(query6)
print(response6)


Cha mẹ đẻ hoặc người giám hộ lập hồ sơ của người được giới thiệu làm con nuôi sống tại gia đình; cơ sở nuôi dưỡng lập hồ sơ của trẻ em được giới thiệu làm con nuôi sống tại cơ sở nuôi dưỡng. Điều 19. Nộp hồ sơ, thời hạn giải quyết việc nuôi con nuôi  1. Người nhận con nuôi phải nộp hồ sơ của mình và hồ sơ của người được giới thiệu làm con nuôi tại Ủy ban nhân dân cấp xã nơi người được giới thiệu làm con nuôi thường trú hoặc nơi người nhận con nuôi thường trú. 2. Thời hạn giải quyết việc nuôi con nuôi là 30 ngày, kể từ ngày Ủy ban nhân dân cấp xã nhận đủ hồ sơ hợp lệ. Điều 20. Kiểm tra hồ sơ, lấy ý kiến của những người liên quan  1. Ủy ban nhân dân cấp xã nơi nhận hồ sơ có trách nhiệm kiểm tra hồ sơ; trong thời hạn mười ngày, kể từ ngày nhận đủ hồ sơ hợp lệ, tiến hành xong việc lập ý kiến của những người quy định tại Điều 21 của Luật này. 2. Trong trường hợp trẻ em mồ côi không có người nuôi dưỡng hoặc trẻ em có cha mẹ đẻ, người thân thích nhưng không có khả năng nuôi dưỡng thì người g

In [ ]:
query6 = "Theo quy định mới, ai có thẩm quyền quyết định xếp hạng di tích quốc gia đặc biệt?"
response6 = answer(query6)
print(response6)

 
Thủ tướng Chính phủ có thẩm quyền ra quyết định xếp hạng di tích quốc gia đặc biệt theo quy định tại điểm c khoản 1 Điều 30 Luật Di sản văn hóa.
Thẩm quyền, thủ tục và hồ sơ trình Thủ tướng Chính phủ để xếp hạng di tích được quy định cụ thể từ Điều 30 đến Điều 32 của Luật Di sản Văn hóa năm 2001.
Theo đó:
- Chủ tịch Ủy ban nhân dân cấp tỉnh quyết định xếp hạng di tích cấp tỉnh;
- Bộ trưởng Bộ Văn hoá - Thông tin quyết định xếp hạng di tích quốc gia;
- Thủ tướng Chính phủ ra quyết định xếp hạng di tích quốc gia đặc biệt. 
